In [1]:
from pathlib import Path
import sys

module_path = Path.cwd().parent.as_posix()
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import simpleaudio as sa
from datetime import datetime as dt
from functools import cache
import random
from time import sleep
import numpy as np

In [3]:
from IPython.display import Audio

wave_audio = np.sin(np.linspace(0, 3000, 5000))
Audio(wave_audio, rate=20000)

## Custom functions for simulating trials

In [4]:
def simulate_buzzer(duration_sec=1):
    """Simulate the buzzer for the specified duration in seconds."""
    print(f"Simulating buzzer for {duration_sec} seconds.")
    # code to make buzzer sound

In [5]:
DATA_DIR = Path.cwd().parent / "data"

In [6]:
def log_event(
    event_name, event_time, trial_number=None, log_file=DATA_DIR / "event_log.txt"
):
    """Log the event name and time to a file."""
    with open(log_file, "a") as f:
        f.write(f"{event_time}: {event_name}\n")
    print(f"Logged - {event_time}: {event_name}\n")
    return None

In [7]:
@cache
def calculate_audio(duration, fs):
    frequency = 880  # Our played note will be 440 Hz
    seconds = duration  # Duration in seconds (must be integer)

    # fs: samples per second
    # Generate array with seconds*sample_rate steps, ranging between 0 and seconds
    t = np.linspace(0, seconds, seconds * fs, False)

    # Generate a 440 Hz sine wave
    note = np.sin(frequency * t * 2 * np.pi)

    # Ensure that highest value is in 16-bit range
    audio = note * (2**15 - 1) / np.max(np.abs(note))
    # Convert to 16-bit data
    audio = audio.astype(np.int16)
    return audio

In [8]:
def play_beep(duration=1, fs=44100):
    """Play a beep sound for the specified duration in seconds."""
    audio = calculate_audio(duration, fs)

    # Start playback
    play_obj = sa.play_buffer(audio, 1, 2, fs)

    # Wait for playback to finish before exiting
    play_obj.wait_done()

In [9]:
# !say please put the horse in the test chute

# TODO: How do you know the horse is in the chute and ready to start the trial?

In [10]:
# Trial parameters

ACTIVATION_TIMEOUT_SEC = 5
WAIT_AFTER_CORRECT_RESPONSE_SEC = 3
FEED_CONSUMPTION_TIMEOUT_SEC = 15
TRIAL_TIMEOUT_SEC = 60

In [11]:
trial_parameters_names = [
    "ACTIVATION_TIMEOUT_SEC",
    "WAIT_AFTER_CORRECT_RESPONSE_SEC",
    "FEED_CONSUMPTION_TIMEOUT_SEC",
    "TRIAL_TIMEOUT_SEC",
]

In [12]:
def log_trial_parameters(
    trial_parameters_names=trial_parameters_names,
    log_file=DATA_DIR / "trial_parameters_log.txt",
):
    trial_parameters = {key: eval(key) for key in trial_parameters_names}

    """Log the trial parameters to a file."""
    with open(log_file, "a") as f:
        f.write(f"TRIAL PARAMETERS: {trial_parameters}\n")
    print(f"Logged - TRIAL PARAMETERS: {trial_parameters}\n")
    return None

In [13]:
def play_correct_response_tone():
    """Play the correct response tone."""
    log_event("Start playing correct response tone", dt.now())
    play_beep(duration=1)
    log_event("Finished playing correct response tone", dt.now())
    return None

In [14]:
def dispense_feed():
    """Dispense feed."""
    log_event("Dispensing feed", dt.now())
    # code to dispense feed
    return None

In [15]:
# Experiment trial

log_trial_parameters()

# Event 0: Horse enters test chute - how does the experimenter know this?

# Event 1: Start tone - 1 sec duration

duration = 1
log_event(f"Playing buzzer for {duration} seconds", dt.now())
play_beep(duration)
log_event("Start buzzer finished", dt.now())

# Event 2: Activation of touch sensor (nose press on panel)

start_sensor_period = dt.now()
activate_touch_sensor = False

while (dt.now() - start_sensor_period).seconds < ACTIVATION_TIMEOUT_SEC:
    sleep(0.9)
    if random.random() < 0.2:  # simulate touch sensor activation
        touch_latency = (dt.now() - start_sensor_period).total_seconds()
        log_event(f"Touch sensor activated after {touch_latency} seconds", dt.now())
        activate_touch_sensor = True
        # reset_touch_sensor()
        play_correct_response_tone()
        log_event(
            f"Waiting after correct response finished for {WAIT_AFTER_CORRECT_RESPONSE_SEC} seconds",
            dt.now(),
        )
        sleep(WAIT_AFTER_CORRECT_RESPONSE_SEC)
        dispense_feed()
        log_event(
            f"Waiting after feed dispensed for {WAIT_AFTER_CORRECT_RESPONSE_SEC} seconds",
            dt.now(),
        )
        sleep(FEED_CONSUMPTION_TIMEOUT_SEC)
        break
if not activate_touch_sensor:
    log_event(
        f"Touch sensor not activated after {ACTIVATION_TIMEOUT_SEC} seconds", dt.now()
    )

# Event 3: Correct response tone - 0.5 sec duration

log_event("Trial ends", dt.now())

Logged - TRIAL PARAMETERS: {'ACTIVATION_TIMEOUT_SEC': 5, 'WAIT_AFTER_CORRECT_RESPONSE_SEC': 3, 'FEED_CONSUMPTION_TIMEOUT_SEC': 15, 'TRIAL_TIMEOUT_SEC': 60}

Logged - 2023-06-22 20:49:30.040907: Playing buzzer for 1 seconds

Logged - 2023-06-22 20:49:31.222672: Start buzzer finished

Logged - 2023-06-22 20:49:34.841700: Touch sensor activated after 3.618547 seconds

Logged - 2023-06-22 20:49:34.843347: Start playing correct response tone

Logged - 2023-06-22 20:49:35.911491: Finished playing correct response tone

Logged - 2023-06-22 20:49:35.912196: Waiting after correct response finished for 3 seconds

Logged - 2023-06-22 20:49:38.917859: Dispensing feed

Logged - 2023-06-22 20:49:38.918545: Waiting after feed dispensed for 3 seconds

Logged - 2023-06-22 20:49:53.924450: Trial ends

